# Modelo de Aprendizaje Automático:
Después de completar el Análisis Exploratorio de Datos (EDA) para comprender la naturaleza de nuestros datos, avanzamos hacia la creación de un modelo de recomendación basado en la similitud del coseno. El enfoque del modelo se fundamenta en la similitud entre juegos. Al proporcionar el nombre o identificador de un juego específico, el sistema generará una lista de 5 juegos recomendados que comparten similitudes con el juego de referencia. Esta metodología es ideal para usuarios que buscan descubrir títulos relacionados o similares a sus juegos preferidos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import scipy.sparse as sp
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import seaborn as sns

## Carga de Datos


In [2]:
df_output = pd.read_csv("data_repo/output_steam_games_limpio.csv")
df_output

,app_name,id,developer,price_numeric,release_year,main_genre
0,Lost Summoner Kitty,761140,Kotoshiro,4.99,2018,Action
1,Ironbound,643980,Secret Level SRL,0.00,2018,Free to Play
2,Real Pool 3D - Poolians,670290,Poolians.com,0.00,2017,Casual
3,弹炸人2222,767400,彼岸领域,0.99,2017,Action
4,Battle Royale Trainer,772540,Trickjump Games Ltd,3.99,2018,Action
...,...,...,...,...,...,...
28829,Kebab it Up!,745400,Bidoniera Games,1.99,2018,Action
28830,Colony On Mars,773640,"Nikita ""Ghost_RUS""",1.99,2018,Casual
28831,LOGistICAL: South Africa,733530,Sacada,4.99,2018,Casual
28832,Russian Roads,610660,Laush Dmitriy Sergeevich,1.99,2018,Indie


## Selección de Columnas Relevantes
En esta sección, se seleccionaron las columnas relevantes ("app_name", "id", y "main_genre") del DataFrame original df_output.

In [3]:
df_recomendacion =df_output[["app_name","id","main_genre"]]
df_recomendacion

,app_name,id,main_genre
0,Lost Summoner Kitty,761140,Action
1,Ironbound,643980,Free to Play
2,Real Pool 3D - Poolians,670290,Casual
3,弹炸人2222,767400,Action
4,Battle Royale Trainer,772540,Action
...,...,...,...
28829,Kebab it Up!,745400,Action
28830,Colony On Mars,773640,Casual
28831,LOGistICAL: South Africa,733530,Casual
28832,Russian Roads,610660,Indie


## Codificación One-Hot
Se aplicó la codificación one-hot a la columna "main_genre" utilizando la función pd.get_dummies.

In [4]:
modelo_recomendacion = pd.get_dummies(df_recomendacion, columns=["main_genre"])
modelo_recomendacion 

,app_name,id,main_genre_Accounting,main_genre_Action,main_genre_Adventure,main_genre_Animation &amp; Modeling,main_genre_Audio Production,main_genre_Casual,main_genre_Design &amp; Illustration,main_genre_Early Access,...,main_genre_RPG,main_genre_Racing,main_genre_Simulation,main_genre_Sin Dato,main_genre_Software Training,main_genre_Sports,main_genre_Strategy,main_genre_Utilities,main_genre_Video Production,main_genre_Web Publishing
0,Lost Summoner Kitty,761140,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,Ironbound,643980,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,Real Pool 3D - Poolians,670290,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
3,弹炸人2222,767400,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,Battle Royale Trainer,772540,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28829,Kebab it Up!,745400,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
28830,Colony On Mars,773640,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
28831,LOGistICAL: South Africa,733530,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
28832,Russian Roads,610660,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## Transformación de Valores Booleanos
Se reemplazaron los valores booleanos por 1 y 0 en el DataFrame modelo_recomendacion.

In [5]:
modelo_recomendacion = modelo_recomendacion.replace({True: 1, False: 0})
modelo_recomendacion

,app_name,id,main_genre_Accounting,main_genre_Action,main_genre_Adventure,main_genre_Animation &amp; Modeling,main_genre_Audio Production,main_genre_Casual,main_genre_Design &amp; Illustration,main_genre_Early Access,...,main_genre_RPG,main_genre_Racing,main_genre_Simulation,main_genre_Sin Dato,main_genre_Software Training,main_genre_Sports,main_genre_Strategy,main_genre_Utilities,main_genre_Video Production,main_genre_Web Publishing
0,Lost Summoner Kitty,761140,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ironbound,643980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Real Pool 3D - Poolians,670290,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,弹炸人2222,767400,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Battle Royale Trainer,772540,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28829,Kebab it Up!,745400,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28830,Colony On Mars,773640,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28831,LOGistICAL: South Africa,733530,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28832,Russian Roads,610660,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Modificación de Nombres de Columnas
Se realizaron modificaciones en los nombres de las columnas para quitar el prefijo "main_genre_".

In [6]:
modelo_recomendacion.columns = modelo_recomendacion.columns.str.replace("main_genre_", "")
modelo_recomendacion

,app_name,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,...,RPG,Racing,Simulation,Sin Dato,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Lost Summoner Kitty,761140,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Ironbound,643980,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Real Pool 3D - Poolians,670290,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,弹炸人2222,767400,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Battle Royale Trainer,772540,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28829,Kebab it Up!,745400,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
28830,Colony On Mars,773640,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28831,LOGistICAL: South Africa,733530,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
28832,Russian Roads,610660,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
modelo_recomendacion.columns

Index(['app_name', 'id', 'Accounting', 'Action', 'Adventure',
       'Animation &amp; Modeling', 'Audio Production', 'Casual',
       'Design &amp; Illustration', 'Early Access', 'Education',
       'Free to Play', 'Indie', 'Massively Multiplayer', 'Photo Editing',
       'RPG', 'Racing', 'Simulation', 'Sin Dato', 'Software Training',
       'Sports', 'Strategy', 'Utilities', 'Video Production',
       'Web Publishing'],
      dtype='object')

In [10]:
modelo_2 = "Datasets/modelo_reco.csv"
modelo_recomendacion.to_csv(modelo_2, index=False, encoding="utf-8")
print(f"Se guardó el archivo {modelo_2}") 

Se guardó el archivo Datasets/modelo_reco.csv


## Función de Recomendación de Juegos
En esta sección se definió una función llamada recomendacion_juego que tiene como objetivo proporcionar recomendaciones de juegos similares en función del ID del juego proporcionado.
* Entrada:
id_juego: El ID del juego para el cual se desea obtener recomendaciones.
* Proceso:
La función verifica si el ID del juego es válido y existe en el DataFrame modelo_recomendacion.
Se realiza un cálculo de similitud coseno entre el juego dado y una muestra aleatoria de juegos.
Se ordenan los juegos en la muestra por similitud y se seleccionan los 5 más similares.
* Salida:
La función devuelve un diccionario con los nombres de los juegos recomendados. Si hay un error, devuelve un mensaje de error correspondiente.

In [8]:
def recomendacion_juego(id_juego):
    try:
        id_juego = int(id_juego)
        juego_seleccionado = modelo_recomendacion[modelo_recomendacion["id"] == id_juego]
        if juego_seleccionado.empty:
            return {"error": f"El juego con el ID '{id_juego}' no se encuentra."}
        indice_juego = juego_seleccionado.index[0]
        muestra = 3000
        df_muestra = modelo_recomendacion.sample(n=muestra, random_state=50)
        juego_features = modelo_recomendacion.iloc[indice_juego, 3:]
        muestra_features = df_muestra.iloc[:, 3:]
        similitud = cosine_similarity([juego_features], muestra_features)[0]
        recomendaciones = sorted(enumerate(similitud), key=lambda x: x[1], reverse=True)[:5]
        recomendaciones_indices = [i[0] for i in recomendaciones]
        recomendaciones_names = df_muestra["app_name"].iloc[recomendaciones_indices].tolist()

        return {"Juegos_similares": recomendaciones_names}

    except ValueError:
        return {"error": "El ID del juego debe ser un número entero válido."}


In [9]:
recomendacion_juego(1250)

{'Juegos_similares': ['Dangerous Golf',
  'Gold Rush In The Oort Cloud',
  "Batman Arkham City: Harley Quinn's Revenge",
  'DARIUSBURST Chronicle Saviours - Night Striker',
  'Hitman 2: Silent Assassin']}